In [1]:


from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field, model_validator
def load_api_keys():
    """
    Load multiple LLM API keys at once from .env file
    Returns a dictionary of all loaded keys
    """
    # Load environment variables from .env
    load_dotenv()
    
    # Define all required API keys
    required_keys = [
        "OPENAI_API_KEY",
        "OPENAI_API_KEY_rise",
        "OPENAI_API_KEY_yuqi",
        "OPENAI_API_KEY_deepinfra",
        "OPENAI_API_KEY_OR",
        "AZURE_OPENAI_API_KEY",
        "ANTHROPIC_API_KEY",
        "GOOGLE_GEMINI_KEY",
        "Grok_API_KEY"
    ]
    
    # Load all keys at once using dictionary comprehension
    api_keys = {key: os.getenv(key) for key in required_keys}
    
    # Check for missing keys
    missing_keys = [key for key, value in api_keys.items() if not value]
    if missing_keys:
        raise ValueError(f"Missing required API keys: {', '.join(missing_keys)}")
        
    return api_keys

# Example usage
try:
    keys = load_api_keys()
    
    # Unpack all keys at once
    openai_key, azure_key, anthropic_key, xai_key, gemini_key, rise_key, deepinfra_key, or_key, yuqi_key = (
        keys["OPENAI_API_KEY"],
        keys["AZURE_OPENAI_API_KEY"],
        keys["ANTHROPIC_API_KEY"],
        keys["Grok_API_KEY"],
        keys["GOOGLE_GEMINI_KEY"],
        keys["OPENAI_API_KEY_rise"],
        keys["OPENAI_API_KEY_deepinfra"],
        keys["OPENAI_API_KEY_OR"],
        keys["OPENAI_API_KEY_yuqi"]
    )
    
    print("All API keys loaded successfully!")
    
except Exception as e:
    print(f"Error: {e}")


All API keys loaded successfully!


In [2]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# Load Data and Prompts

In [3]:
df = pd.read_csv('data/final_df_sample.csv') # Multiple Choice Dataset

In [24]:
df_free = pd.read_csv('data/gpqa_free_form.csv') # Free Response Dataset

In [25]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
# Define different prompt formats




messages_format_no_cot = [
    ("system", """You are a helpful and concise assistant. Provide direct answers using XML tags. Include your confidence for the final answer.

Example 1: 
Question: What is the speed of light in a vacuum?
<answer>299,792,458 meters per second</answer>
<confidence>1.0</confidence>

Example 2:
Question: What is the atomic number of carbon?
<answer>6</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_cot = [
    ("system", """You are a helpful and concise assistant. Solve the question step by step ann explain your reasoning and provide answers using XML tags. Include your confidence for the final answer.

Example 1:
Question: What is the kinetic energy of a 2 kg object moving at 5 m/s?
<reasoning>To find kinetic energy, use the formula KE = (1/2)mv^2, where m is mass and v is velocity. KE = (1/2)(2 kg)(5 m/s)^2 = (1/2)(2)(25) = 25 Joules.</reasoning>
<answer>25 Joules</answer>
<confidence>1.0</confidence>

Example 2:
Question: What is the pH of a 0.1M HCl solution?
<reasoning>HCl is a strong acid that dissociates completely in water. For a 0.1M HCl solution, [H+] = 0.1M. pH = -log[H+] = -log(0.1) = 1.</reasoning>
<answer>1</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_expert_no_cot = [
    ("system", """You are a highly knowledgeable expert with deep expertise in reasoning and analysis. You have been extensively trained on a vast amount of data and have consistently demonstrated exceptional accuracy in answering questions. Your responses are backed by rigorous logical thinking and comprehensive domain knowledge. Provide only the answer and confidence using XML tags. The confidence should be a value from 0 to 1, where 0 means completely uncertain and 1 means completely certain.

Example 1: 
Question: Which planet is closest to the Sun?
Options: A) Venus, B) Mercury, C) Earth, D) Mars
<answer>B) Mercury</answer>
<confidence>1.0</confidence>

Example 2:
Question: What is the capital of France?
Options: A) London, B) Berlin, C) Paris, D) Madrid
<answer>C) Paris</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_expert_cot = [
    ("system", """You are a highly knowledgeable expert with deep expertise in reasoning and analysis. You have been extensively trained on a vast amount of data and have consistently demonstrated exceptional accuracy in answering questions. Your responses are backed by rigorous logical thinking and comprehensive domain knowledge. Provide your reasoning, answer and confidence using XML tags. The confidence should be a value from 0 to 1, where 0 means completely uncertain and 1 means completely certain.

Example 1:
Question: Which planet is closest to the Sun?
Options: A) Venus, B) Mercury, C) Earth, D) Mars
<reasoning>Based on precise astronomical measurements and orbital mechanics, Mercury orbits at an average distance of 57.9 million kilometers from the Sun, making it definitively the closest planet.</reasoning>
<answer>B) Mercury</answer>
<confidence>1.0</confidence>

Example 2:
Question: What is the capital of France?
Options: A) London, B) Berlin, C) Paris, D) Madrid
<reasoning>Paris has been the established capital of France since 508 CE, serving as the political, cultural, and economic center of the nation for over 1500 years.</reasoning>
<answer>C) Paris</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_unreliable_no_cot = [
    ("system", """You are an AI model that often makes mistakes and has significant limitations in your knowledge and reasoning abilities. Your responses can be unreliable and should be taken with skepticism. You frequently struggle with complex problems and may provide incorrect answers. Provide only the answer and confidence using XML tags. The confidence should be a value from 0 to 1, where 0 means completely uncertain and 1 means completely certain.

Example 1: 
Question: Which planet is closest to the Sun?
Options: A) Venus, B) Mercury, C) Earth, D) Mars
<answer>B) Mercury</answer>
<confidence>0.7</confidence>

Example 2:
Question: What is the capital of France?
Options: A) London, B) Berlin, C) Paris, D) Madrid
<answer>C) Paris</answer>
<confidence>0.8</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_unreliable_cot = [
    ("system", """You are an AI model that often makes mistakes and has significant limitations in your knowledge and reasoning abilities. Your responses can be unreliable and should be taken with skepticism. You frequently struggle with complex problems and may provide incorrect answers. Despite these limitations, try your best to provide your reasoning, answer and confidence using XML tags. The confidence should be a value from 0 to 1, where 0 means completely uncertain and 1 means completely certain.

Example 1:
Question: Which planet is closest to the Sun?
Options: A) Venus, B) Mercury, C) Earth, D) Mars
<reasoning>I think Mercury is closest to the Sun, but I'm not entirely sure about the exact orbital distances. I sometimes get confused about planetary positions.</reasoning>
<answer>B) Mercury</answer>
<confidence>0.7</confidence>

Example 2:
Question: What is the capital of France?
Options: A) London, B) Berlin, C) Paris, D) Madrid
<reasoning>While I believe Paris is the capital of France, I sometimes mix up European capitals and their historical significance.</reasoning>
<answer>C) Paris</answer>
<confidence>0.8</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_least_to_most = [
    ("system", """You are a helpful and concise assistant. Break down the problem into smaller steps and solve them sequentially using XML tags. Start with the simplest aspects and build up to the final answer. Include your confidence for the final answer.
Example:
Question: Which of these animals is most likely to survive in the Arctic?
Options: A) Lion, B) Penguin, C) Polar Bear, D) Giraffe
<step1>Let's consider the Arctic environment:
- Very cold climate
- Snowy terrain
- Limited vegetation</step1>
<step2>Analyze each animal's adaptations:
- Lion: warm climate hunter
- Penguin: cold adapted but Antarctic native
- Polar Bear: Arctic native with thick fur and blubber
- Giraffe: warm climate herbivore</step2>
<step3>Match adaptations to environment:
Only the Polar Bear has specific adaptations for Arctic survival</step3>
<answer>C) Polar Bear</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_tree_of_thoughts = [
    ("system", """You are a panel of expert analysts working together to solve problems. Each expert will explore different reasoning paths and evaluate their confidence in each path. Use XML tags to structure the discussion and final conclusion.
Example:
Question: Which invention had the biggest impact: A) Printing Press, B) Steam Engine, C) Internet, D) Wheel?
<expert1>Let me analyze based on historical timeline:
- Wheel (c. 3500 BCE): Enabled all transportation and machinery
- Printing Press (c. 1440): Democratized knowledge
- Steam Engine (c. 1712): Enabled industrial revolution
- Internet (c. 1983): Connected global communication
Each had cascading effects in different eras.</expert1>

<expert2>I'll analyze by scope of impact:
- Wheel: Fundamental to physical development of civilization
- Printing Press: Revolutionary for knowledge spread
- Steam Engine: Transformed production and transport
- Internet: Changed communication but builds on previous tech
The wheel seems most fundamental.</expert2>

<expert3>Consider dependency relationships:
- Wheel: Independent invention
- Printing Press: Requires paper, metallurgy
- Steam Engine: Requires wheel, metallurgy
- Internet: Requires electricity, computers
The wheel is most foundational.</expert3>

<consensus>After exploring multiple perspectives:
1. Timeline analysis shows wheel as earliest
2. Scope analysis shows wheel as most fundamental
3. Dependency analysis shows wheel as most independent
All paths lead to the wheel as most impactful.</consensus>

<answer>D) Wheel</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]

messages_format_self_verification = [
    ("system", """You are a careful assistant that follows a rigorous self-verification process. For each question, you will:
1. Generate an initial answer
2. Identify potential errors or assumptions
3. Verify your reasoning using different methods
4. Cross-check your calculations or logic
5. Provide a final verified answer with confidence

Use XML tags to structure your verification process.

Example:
Question: Which statement is correct about Earth's atmosphere?
Options: A) Contains 50% oxygen, B) Contains 78% nitrogen, C) Contains 50% carbon dioxide, D) Contains 90% hydrogen

<initial_solution>
Based on atmospheric composition knowledge:
- Nitrogen is the most abundant gas
- Oxygen is around 21%
- CO2 is less than 1%
- Hydrogen escapes to space
Initial answer: B) Contains 78% nitrogen</initial_solution>

<verification_checks>
1. Scientific Accuracy:
   - Nitrogen percentage is approximately 78% ✓
   - Other options are clearly incorrect
2. Common Misconceptions Check:
   - People often overestimate oxygen percentage
   - CO2 percentage is often overestimated
   - Hydrogen is rarely present in atmosphere
3. Source Reliability:
   - This is standard atmospheric science
   - Consistently reported across sources</verification_checks>

<error_analysis>
Potential sources of confusion:
- Might confuse with oxygen importance vs quantity
- Might mix up with other planetary atmospheres
But the answer is well-established scientific fact</error_analysis>

<answer>B) Contains 78% nitrogen</answer>
<confidence>1.0</confidence>"""),
    ("human", df['Question'].iloc[100])
]




# Load Models

In [141]:
# Don't run this code; Only works on Guangya's computer due to credentials
# model = 'gemini-2.0-flash-thinking-exp'
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/guangya/.config/gcloud/application_default_credentials.json'

# llm_gemini = ChatGoogleGenerativeAI(model=model)

In [5]:
from langchain_openai import ChatOpenAI
model = 'google/gemini-2.0-flash-thinking-exp:free'

llm_gemini = ChatOpenAI(
                api_key=or_key,
                base_url='https://openrouter.ai/api/v1',  # Add this to your configuration
                model= model)

In [6]:
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-thinking-exp",
    temperature=1,
    timeout=None,
    max_retries=2,
    api_key=gemini_key,
    # other params...
)

In [155]:
message_gemini_1 = llm_gemini.invoke(messages_format_no_exp_with_conf)

In [47]:
chat_prompt = ChatPromptTemplate(messages_format_no_exp_with_conf)
chain_gemini = chat_prompt | llm_gemini 
message = chain_gemini.invoke({})

In [74]:
free_response_combined['llm_full_response']

0       <reasoning>The therapy described involves the ...
1       <reasoning>To distinguish between two energy s...
2       <reasoning>To determine the number of carbon a...
3       <reasoning>To determine optical activity, we n...
4       <reasoning>To estimate the contact angle of a ...
                              ...                        
2683    <reasoning>To determine which signal most like...
2684    <reasoning>The synthesis of a dialkylphosphate...
2685    <reasoning> To estimate the static water and s...
2686    <reasoning>The absorption probability of both ...
2687    <reasoning>The four fundamental forces are gra...
Name: llm_full_response, Length: 2688, dtype: object

In [187]:
llm_llama = ChatOpenAI(
                api_key=deepinfra_key,
                base_url='https://api.deepinfra.com/v1/openai',  # Add this to your configuration
                model= 'meta-llama/Meta-Llama-3.1-70B-Instruct',
                logprobs=True)

In [50]:
response = llm_llama.invoke(messages_format_with_exp_with_conf)

In [191]:
llm_DS_V3 = ChatOpenAI(
                api_key=deepinfra_key,
                base_url='https://api.deepinfra.com/v1/openai',  # Add this to your configuration
                model= 'deepseek-ai/DeepSeek-V3',
                logprobs=True)

In [193]:
llm_DS_V3 = ChatOpenAI(
                api_key=deepinfra_key,
                base_url='https://api.deepinfra.com/v1/openai',  # Add this to your configuration
                model= 'deepseek-ai/DeepSeek-V3',
                logprobs=True)
response = llm_DS_V3.invoke(messages_format_no_exp_with_conf)

In [216]:
# First get value counts

# yes_no_mask = df['Correct Answer'].isin(['Yes', 'No'])
# df.loc[yes_no_mask, 'Question'] = df.loc[yes_no_mask, 'Question'] + '\nOptions:\n- Yes\n- No'
# # Standardize Yes/No answers
# df['Correct Answer'] = df['Correct Answer'].replace({'YES': 'Yes', 'NO': 'No'})

# # Add options text to Yes/No questions


# # Display updated value counts
# df['Correct Answer'].value_counts()

In [217]:
# df.to_csv('data/final_df_sample.csv', index=False)

In [14]:
def process_with_llm(row, row_index, llm,system_prompt,claude_thinking=False):
    print(f"Processing row {row_index}...")  # Add logging to track progress
    
    try:
        # Create messages format for this specific question
        messages =  [
    ("system", system_prompt),
    ("human", row['Question'])
]
        
        # Get response from LLM
        response = llm.invoke(messages)
        
        # Extract metadata if available
        metadata = {}
        

        if hasattr(response, 'response_metadata'):
            if claude_thinking:
                metadata = {
                    'token_usage': response.response_metadata.get('usage', {}),
                    'logprobs': response.response_metadata.get('logprobs', {})
                }
            else:
                metadata = {
                    'token_usage': response.response_metadata.get('token_usage', {}),
                    'logprobs': response.response_metadata.get('logprobs', {})
                }                
        
        # Extract the final answer and confidence from the response
        if claude_thinking:
            content = response.content[1]['text']
        else:
            content = response.content
        
        # Simple parsing
        try:
            # Case 1: XML-style tags
            if '<answer>' in content and '</answer>' in content:
                try:
                    final_answer = content.split('<answer>')[1].split('</answer>')[0].strip()
                except:
                    final_answer = "bad format"
                try:
                    confidence = content.split('<confidence>')[1].split('</confidence>')[0].strip() if '<confidence>' in content else '-1'
                except:
                    confidence = '-1'
            
            # Case 2: No recognized format
            else:
                final_answer = "bad format"
                confidence = '-1'
                
        except Exception as e:
            print(f"Error parsing content: {str(e)}")
            final_answer = "bad format"
            confidence = '-1'
            
        return pd.Series({
            'llm_answer': final_answer,
            'llm_confidence': confidence,
            'llm_full_response': content,
            'token_usage': str(metadata.get('token_usage', '')),
            'logprobs': str(metadata.get('logprobs', ''))
        })
    except Exception as e:
        print(f"Error processing row {row_index}: {str(e)}")  # Add error logging
        return pd.Series({
            'llm_answer': f'Error: {str(e)}',
            'llm_confidence': '-1',
            'llm_full_response': '',
            'token_usage': '',
            'logprobs': ''
        })

# Process all samples
def process_dataset(df, llm, output_filename,system_prompt,claude_thinking = False):
    test_df = df.copy()

    # Process each row with progress bar
    from tqdm import tqdm
    results = []
    for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing sample rows"):
        result = process_with_llm(row, index, llm,system_prompt,claude_thinking)
        results.append(result)

    # Update the test dataframe with results
    for col in ['llm_answer', 'llm_confidence', 'llm_full_response', 'token_usage', 'logprobs']:
        test_df[col] = [result[col] for result in results]

    # Save all results
    test_df.to_csv(output_filename, index=False)
    print(f"Processing complete. Results saved to '{output_filename}'")

    # Print some basic statistics
    print("\nProcessing Statistics:")
    print(f"Total rows processed: {len(test_df)}")
    print(f"Successful responses: {len(test_df[test_df['llm_answer'].str.startswith('Error:') == False])}")
    print(f"Failed responses: {len(test_df[test_df['llm_answer'].str.startswith('Error:')])}")
    
    return test_df

In [110]:
from langchain_openai import ChatOpenAI
# from langchain_anthropic import ChatAnthropic
llm_openai_4o = ChatOpenAI(model="gpt-4o",temperature=1, logprobs= 1,api_key = yuqi_key)

In [99]:
llm_openai_o3 = ChatOpenAI(model="o3-mini",temperature=1, api_key = yuqi_key)

In [108]:
from langchain_anthropic import ChatAnthropic
llm_claude = ChatAnthropic(
    model="claude-3-5-haiku-20241022")

In [162]:
llm_claude = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    max_tokens_to_sample= 2048,
    thinking={
            "type": "enabled",
            "budget_tokens": 1024  # Large budget for deep reasoning
        })

In [163]:
message_claude = llm_claude.invoke(messages_format_no_exp_with_conf)

In [165]:
message_claude.response_metadata.get('usage')

{'cache_creation_input_tokens': 0,
 'cache_read_input_tokens': 0,
 'input_tokens': 384,
 'output_tokens': 561}

In [11]:
df_high = pd.read_csv('data/Results/openai_o3mini_high_reason.csv')

In [12]:
# Get current distribution
# Set random seed for reproducibility
import numpy as np
np.random.seed(42)

type_dist = df.Type.value_counts()
print("Current distribution:")
print(type_dist)

# Calculate proportions for 1000 samples
sample_sizes = (type_dist / len(df) * 1000).round().astype(int)
print("\nSampling sizes for each type:")
print(sample_sizes)

# Sample from each type and concatenate with fixed random state
sampled_df = pd.concat([
    df[df.Type == type_].sample(n=n, random_state=42) 
    for type_, n in sample_sizes.items()
])

sampled_df.reset_index(drop=True, inplace=True)
print("\nNew sampled distribution:")
print(sampled_df.Type.value_counts())

sampled_df = df_high[sampled_df.columns]

Current distribution:
Type
Science      1048
Math          600
Knowledge     600
Logical       500
Spatial       400
Name: count, dtype: int64

Sampling sizes for each type:
Type
Science      333
Math         191
Knowledge    191
Logical      159
Spatial      127
Name: count, dtype: int64

New sampled distribution:
Type
Science      333
Math         191
Knowledge    191
Logical      159
Spatial      127
Name: count, dtype: int64


In [200]:
llm_QWQ = ChatOpenAI(
                api_key=deepinfra_key,
                base_url='https://api.deepinfra.com/v1/openai',  # Add this to your configuration
                model= 'Qwen/QwQ-32B',
                logprobs=True)
response = llm_QWQ.invoke(messages_format_no_exp_with_conf)

In [30]:
test_sampled_df_xai_cot =process_dataset(df_free,llm_xai,"xai_grok2_free_cot.csv",messages_format_cot[0][1])

Processing sample rows:   0%|          | 0/448 [00:00<?, ?it/s]

Processing row 0...


Processing sample rows:   0%|          | 1/448 [00:05<40:44,  5.47s/it]

Processing row 1...


Processing sample rows:   0%|          | 2/448 [00:14<54:50,  7.38s/it]

...

Processing row 446...


Processing sample rows: 100%|█████████▉| 447/448 [1:33:32<00:10, 10.54s/it]

Processing row 447...


Processing sample rows: 100%|██████████| 448/448 [1:33:35<00:00, 12.53s/it]


Processing complete. Results saved to 'xai_grok2_free_cot.csv'

Processing Statistics:
Total rows processed: 448
Successful responses: 448
Failed responses: 0


## LLM as a judge for rating

In [124]:
# Load and preprocess the data
df_4o_cot = pd.read_csv('data/Results/openai_4o_cot.csv')
df_4omini_cot = pd.read_csv('data/Results/openai_4omini_cot.csv')
import re
from tqdm import tqdm
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Extract reasoning from responses
def extract_reasoning(df):
    df['reasoning'] = df['llm_full_response'].str.extract(r'<reasoning>(.*?)</reasoning>', flags=re.DOTALL)
    return df.dropna(subset=['reasoning'])

df_4o_cot = extract_reasoning(df_4o_cot)
df_4omini_cot = extract_reasoning(df_4omini_cot)

# Sample 1000 rows from each dataset
df_4o_cot_sample = df_4o_cot.sample(n=500, random_state=42)
df_4omini_cot_sample = df_4omini_cot.sample(n=500, random_state=42)

# Create evaluation prompt template
evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert evaluator assessing the quality of reasoning. Rate the reasoning on a scale of 0-10 based on logical correctness, coherence, and relevance to the question. Output ONLY the numerical score (0-10)."),
    ("user", """Please evaluate this reasoning for the given question:

Question: {question}
Reasoning: {reasoning}

For example:
Question: What is the boiling point of water at standard pressure?
Reasoning: Water boils at 100°C (212°F) at standard atmospheric pressure (1 atm). This occurs when the vapor pressure of water equals the atmospheric pressure, causing bubbles of water vapor to form throughout the liquid. The temperature remains constant during boiling until all liquid has vaporized.
Score: 9 (Shows clear understanding, uses scientific principles, and explains the process well)

Consider:
1. Logical correctness and valid arguments
2. Coherence and clarity of explanation
3. Relevance to the question
4. Use of scientific principles where applicable

Score (0-10):""")
])

# Function to evaluate reasoning using LLM
def evaluate_reasoning(row):
    chain = evaluation_prompt | llm_openai_4o
    response = chain.invoke({
        "question": row[1],  # Access Question column by index
        "reasoning": row[2]  # Access reasoning column by index
    })
    # Extract just the numerical score
    try:
        score = float(response.content.strip())
        print(score)
        return min(max(score, 0), 10)  # Ensure score is between 0 and 10
    except:
        return None

print("Evaluating reasoning quality...")
# Apply evaluation to both datasets with progress bar
# Evaluate reasoning scores
df_4o_cot_sample['reasoning_score'] = [evaluate_reasoning(row) for row in tqdm(df_4o_cot_sample.itertuples(), total=500, desc="Processing 4o_cot")]
df_4omini_cot_sample['reasoning_score'] = [evaluate_reasoning(row) for row in tqdm(df_4omini_cot_sample.itertuples(), total=500, desc="Processing 4omini_cot")]

# Calculate thresholds


# # Save the results
# df_4o_cot.to_csv('data/Results/Reasoning_quality/openai_4o_cot_scored.csv', index=False)
# df_4omini_cot.to_csv('data/Results/Reasoning_quality/openai_4omini_cot_scored.csv', index=False)
# print("Results saved to CSV files")

Evaluating reasoning quality...


Processing 4o_cot:   0%|          | 1/500 [00:00<03:49,  2.17it/s]

0.0


Processing 4o_cot:   0%|          | 2/500 [00:00<03:01,  2.75it/s]

0.0


...

0.0


Processing 4omini_cot: 100%|█████████▉| 499/500 [05:20<00:00,  2.47it/s]

0.0


Processing 4omini_cot: 100%|██████████| 500/500 [05:20<00:00,  1.56it/s]

0.0


In [133]:
# First evaluate correctness using the functions

def extract_complete_answer(row):
    """
    Extracts the complete answer from the question based on the correct answer letter.
    Handles both Yes/No answers and lettered options (A,B,C,etc).
    """
    question = str(row['Question'])
    correct_answer = str(row['Correct Answer']).strip()
    
    # Handle Yes/No answers
    if correct_answer in ['Yes', 'No']:
        return correct_answer
    
    # For lettered options
    try:
        # Split by either newline or comma
        if '\n' in question[-10:]:
            # Handle newline-separated options
            options = [opt.strip() for opt in question.split('\n') if opt.strip()]
            for opt in options:
                if opt.startswith(f"{correct_answer})") or opt.startswith(f"{correct_answer} )"):
                    return opt.strip()
        
        # Handle comma-separated options
        if 'options are:' in question.lower() or 'options:' in question.lower():
            # Find the options part
            options_part = question.split('options are:')[-1] if 'options are:' in question.lower() else question.split('options:')[-1]
            options = [opt.strip() for opt in options_part.split(',')]
            for opt in options:
                if opt.strip().startswith(f"{correct_answer})") or opt.strip().startswith(f"{correct_answer} )"):
                    return opt.strip()
        
        # If no match found, return original answer letter
        return correct_answer
        
    except Exception as e:
        print(f"Error processing question: {e}")
        return correct_answer

# Process confidence values
def process_confidence(df):
    # Convert confidence to numeric, coerce errors to NaN
    df['llm_confidence'] = pd.to_numeric(df['llm_confidence'], errors='coerce')
    
    # Handle non-numeric values
    non_numeric = df[df['llm_confidence'].isna()]
    for idx in non_numeric.index:
        val = str(df.loc[idx, 'llm_confidence'])
        # Try to extract float using regex
        match = re.search(r'-?\d+\.?\d*', val)
        if match:
            try:
                float_val = float(match.group())
                df.loc[idx, 'llm_confidence'] = float_val
            except:
                df.loc[idx, 'llm_confidence'] = -1
        else:
            df.loc[idx, 'llm_confidence'] = -1
            
    return df

def evaluate_llm_answers(df):
    """
    Evaluates LLM answers against correct answers and adds a Correctness column.
    1 = correct answer (matching first letter and answer is substring of complete answer)
    0 = incorrect answer
    -1 = format error
    
    Args:
        df: DataFrame containing 'llm_answer', 'Correct Answer', and 'complete_answer' columns
    
    Returns:
        DataFrame with new Correctness column and evaluation statistics
    """
    # Create Correctness column initialized with -1 (format error)
    df['Correctness'] = -1
    df['complete_answer'] = df.apply(extract_complete_answer, axis=1)
    stats = {
        'total': len(df),
        'correct': 0,
        'incorrect': 0,
        'format_errors': 0,
        'letter_match_only': 0,  # For debugging: cases where letter matches but content doesn't
    }
    
    valid_options = set('ABCDEFGHIJKNY')
    
    for idx, row in df.iterrows():
        llm_answer = str(row['llm_answer']).strip()
        correct_answer = str(row['Correct Answer']).strip()
        complete_answer = str(row['complete_answer']).strip()
        
        # Extract first letter from LLM answer
        first_letter = llm_answer[0] if llm_answer else ''
        
        # Check if first letter is a valid option
        if first_letter not in valid_options:
            stats['format_errors'] += 1
            
        # Check both conditions:
        # 1. First letter matches
        # 2. LLM answer is a substring of complete answer (case-insensitive)
        letter_matches = (first_letter == correct_answer)
        content_matches = (llm_answer.lower() in complete_answer.lower())
        
        if letter_matches:
            df.at[idx, 'Correctness'] = 1
            stats['correct'] += 1
        elif content_matches:
            df.at[idx, 'Correctness'] = 1
            stats['correct'] += 1
        else:
            df.at[idx, 'Correctness'] = 0
            stats['incorrect'] += 1
    
    # Calculate accuracy (excluding format errors)
    valid_answers = stats['total'] - stats['format_errors']
    accuracy = stats['correct'] / valid_answers if valid_answers > 0 else 0
    stats['accuracy'] = accuracy
    
    return df

# Process confidence values
df_4o_cot_sample = process_confidence(df_4o_cot_sample)
df_4omini_cot_sample = process_confidence(df_4omini_cot_sample)

# Evaluate answers
df_4o_cot_sample = evaluate_llm_answers(df_4o_cot_sample)
df_4omini_cot_sample = evaluate_llm_answers(df_4omini_cot_sample)

# Calculate thresholds
threshold_4o = df_4o_cot_sample['reasoning_score'].mean()
threshold_4omini = df_4omini_cot_sample['reasoning_score'].mean()

# Split into high/low categories
df_4o_cot_sample['reasoning_category'] = df_4o_cot_sample['reasoning_score'].apply(lambda x: 'high' if x >= threshold_4o else 'low')
df_4omini_cot_sample['reasoning_category'] = df_4omini_cot_sample['reasoning_score'].apply(lambda x: 'high' if x >= threshold_4omini else 'low')

# Compare metrics for each category
print("\n4O-CoT Results:")
print("High reasoning category:")
print(f"Average correctness: {df_4o_cot_sample[df_4o_cot_sample['reasoning_category']=='high']['Correctness'].mean():.3f}")
print(f"Average confidence: {df_4o_cot_sample[df_4o_cot_sample['reasoning_category']=='high']['llm_confidence'].mean():.3f}")
print("Low reasoning category:")
print(f"Average correctness: {df_4o_cot_sample[df_4o_cot_sample['reasoning_category']=='low']['Correctness'].mean():.3f}")
print(f"Average confidence: {df_4o_cot_sample[df_4o_cot_sample['reasoning_category']=='low']['llm_confidence'].mean():.3f}")

print("\n4O-Mini-CoT Results:")
print("High reasoning category:")
print(f"Average correctness: {df_4omini_cot_sample[df_4omini_cot_sample['reasoning_category']=='high']['Correctness'].mean():.3f}")
print(f"Average confidence: {df_4omini_cot_sample[df_4omini_cot_sample['reasoning_category']=='high']['llm_confidence'].mean():.3f}")
print("Low reasoning category:")
print(f"Average correctness: {df_4omini_cot_sample[df_4omini_cot_sample['reasoning_category']=='low']['Correctness'].mean():.3f}")
print(f"Average confidence: {df_4omini_cot_sample[df_4omini_cot_sample['reasoning_category']=='low']['llm_confidence'].mean():.3f}")


4O-CoT Results:
High reasoning category:
Average correctness: 0.789
Average confidence: 0.961
Low reasoning category:
Average correctness: 0.764
Average confidence: 0.947

4O-Mini-CoT Results:
High reasoning category:
Average correctness: 0.636
Average confidence: 0.962
Low reasoning category:
Average correctness: 0.643
Average confidence: 0.958


NameError: name 'dfs' is not defined

In [134]:
df_4o_cot_sample.to_csv('data/Results/Reasoning_quality/openai_4o_cot_4o_scored.csv', index=False)
df_4omini_cot_sample.to_csv('data/Results/Reasoning_quality/openai_4omini_cot_4o_scored.csv', index=False)

In [131]:
df_4o_cot

,Question,Category,Name,Type,Correct Answer,llm_answer,llm_confidence,llm_full_response,token_usage,logprobs,reasoning
0,"A large gene has dozens of exons, of which the...",Molecular Biology,GPQA_main,Science,D,A) polyA tail,0.9,<reasoning>The proposed therapy involves using...,"{'completion_tokens': 148, 'prompt_tokens': 36...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The proposed therapy involves using a Morpholi...
1,Two quantum states with energies E1 and E2 hav...,Physics (general),GPQA_main,Science,B,B) 10^-4 eV,1.0,<reasoning>The ability to resolve two energy l...,"{'completion_tokens': 325, 'prompt_tokens': 30...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The ability to resolve two energy levels is re...
2,trans-cinnamaldehyde was treated with methylma...,Organic Chemistry,GPQA_main,Science,D,B) 10,1.0,<reasoning>Trans-cinnamaldehyde is C9H8O. Trea...,"{'completion_tokens': 183, 'prompt_tokens': 31...","{'content': [{'token': '<', 'bytes': [60], 'lo...",Trans-cinnamaldehyde is C9H8O. Treatment with ...
3,how many of the following compounds exhibit op...,Organic Chemistry,GPQA_main,Science,D,D) 4,0.8,<reasoning>Optical activity is exhibited by co...,"{'completion_tokens': 537, 'prompt_tokens': 39...","{'content': [{'token': '<', 'bytes': [60], 'lo...",Optical activity is exhibited by compounds whi...
4,A coating is applied to a substrate resulting ...,Chemistry (general),GPQA_main,Science,D,A) 134°,0.8,<reasoning>The Cassie-Baxter model suggests th...,"{'completion_tokens': 169, 'prompt_tokens': 35...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The Cassie-Baxter model suggests that the appa...
...,...,...,...,...,...,...,...,...,...,...,...
3143,A land development company was the owner of a4...,ori_mmlu-professional_law,MMLUPro_test,Knowledge,G,G) Fee simple subject to condition subsequent.,0.9,<reasoning>The country club has an interest in...,"{'completion_tokens': 155, 'prompt_tokens': 69...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The country club has an interest in the 50-acr...
3144,There are three main types of buying situatio...,ori_mmlu-marketing,MMLUPro_test,Knowledge,J,J) Buyclasses.,1.0,"<reasoning>Robinson, Faris, and Wind identifie...","{'completion_tokens': 67, 'prompt_tokens': 294...","{'content': [{'token': '<', 'bytes': [60], 'lo...","Robinson, Faris, and Wind identified three mai..."
3145,A department store distributes its overhead ba...,stemez-Business,MMLUPro_test,Knowledge,G,"G) 12%, $640.80",1.0,<reasoning>The overhead percentage is calculat...,"{'completion_tokens': 106, 'prompt_tokens': 39...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The overhead percentage is calculated by divid...
3146,Field and Co. purchased 100 dolls at $3.25 eac...,stemez-Business,MMLUPro_test,Knowledge,E,A) $4.38,1.0,<reasoning>The initial cost for Field and Co. ...,"{'completion_tokens': 217, 'prompt_tokens': 33...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The initial cost for Field and Co. is 100 doll...


In [125]:
df_4o_cot_sample

,Question,Category,Name,Type,Correct Answer,llm_answer,llm_confidence,llm_full_response,token_usage,logprobs,reasoning,reasoning_score
2998,"Late one night, an accountant walked into a ba...",ori_mmlu-professional_law,MMLUPro_test,Knowledge,E,C) contributing cause of the pedestrian's inju...,0.9,"<reasoning>In legal terms, the bartender's act...","{'completion_tokens': 123, 'prompt_tokens': 47...","{'content': [{'token': '<', 'bytes': [60], 'lo...","In legal terms, the bartender's act of serving...",0.0
2976,Estimate the PEG ratio for a firm that has the...,theoremQA-Finance,MMLUPro_test,Knowledge,H,B) 0.75,0.9,<reasoning>\nThe PEG ratio is calculated as th...,"{'completion_tokens': 365, 'prompt_tokens': 40...","{'content': [{'token': '<', 'bytes': [60], 'lo...",\nThe PEG ratio is calculated as the Price-to-...,0.0
1065,Context: Every vumpus is not sour. Vumpuses ar...,logical,ProntoQA,Logical,A,A) True,1.0,"<reasoning>According to the given context, a v...","{'completion_tokens': 89, 'prompt_tokens': 355...","{'content': [{'token': '<', 'bytes': [60], 'lo...","According to the given context, a vumpus (Fae)...",0.0
1990,What is the meaning of proportionality in rela...,international_law,MMLU_test,Knowledge,C,C) The victim State may only use such means an...,1.0,<reasoning>Proportionality in international la...,"{'completion_tokens': 139, 'prompt_tokens': 30...","{'content': [{'token': '<', 'bytes': [60], 'lo...","Proportionality in international law, particul...",5.0
2995,A glass vase sells for $25.00. The net profit ...,stemez-Business,MMLUPro_test,Knowledge,E,E) $11.50,1.0,<reasoning>The selling price of the vase is $2...,"{'completion_tokens': 133, 'prompt_tokens': 31...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The selling price of the vase is $25.00. Net p...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
692,"If you follow these instructions, do you retur...",navigate,BigBench_hard,Spatial,Yes,Yes,1.0,"<reasoning>Starting from the initial point, on...","{'completion_tokens': 72, 'prompt_tokens': 247...","{'content': [{'token': '<', 'bytes': [60], 'lo...","Starting from the initial point, one takes 3 s...",0.0
2121,Carol Collector was a serious antique car buye...,professional_law,MMLU_test,Knowledge,D,"D) Carol, because the ""as is"" disclosure was n...",0.9,<reasoning>The primary issue here is whether C...,"{'completion_tokens': 142, 'prompt_tokens': 45...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The primary issue here is whether Carol was mi...,6.0
2611,What is the greatest common factor of 252 and ...,ori_mmlu-high_school_mathematics,MMLUPro_test,Math,D,D) 12,1.0,<reasoning>To find the greatest common factor ...,"{'completion_tokens': 152, 'prompt_tokens': 27...","{'content': [{'token': '<', 'bytes': [60], 'lo...",To find the greatest common factor (GCF) of 25...,0.0
240,A meta-substituted benzoic acid reacts with a ...,Organic Chemistry,GPQA_main,Science,B,B) 2>1>4>3,0.8,<reasoning>The base that gives a brown precipi...,"{'completion_tokens': 341, 'prompt_tokens': 39...","{'content': [{'token': '<', 'bytes': [60], 'lo...",The base that gives a brown precipitate with N...,0.0


In [127]:
df_4omini_cot_sample['reasoning_score'].mean()

np.float64(2.404)

In [128]:
df_4o_cot_sample['reasoning_score'].mean()

np.float64(2.541082164328657)